In [ ]:
import os
import glob
import json
import geojson
from osgeo import gdal

In [ ]:
year = "2023"
dl_folder = os.path.join("./Sentinel-1", year)

def json_latlon_correction(json_file):
    json_file = json.loads(json_file)
    for i in range(len(json_file["coordinates"][0])):
        coord_pair = json_file["coordinates"][0][i]
        json_file["coordinates"][0][i] = coord_pair[::-1]
    return json_file

def bbox(poly):
    coord = poly['geometry']['coordinates']
    xmin = min([i[0] for i in coord[0]])
    ymin = min([i[1] for i in coord[0]])
    xmax = max([i[0] for i in coord[0]])
    ymax = max([i[1] for i in coord[0]])
    return [xmin, ymin, xmax, ymax]

extent_file = "../geodata/32vnm.geojson"

with open(extent_file) as f:
    gj = geojson.load(f)
extentSRS = "EPSG:" + gj['crs']['properties']['name'].split(':')[-1]
extent = gj['features'][0]
print(extentSRS)
envelope = bbox(extent)
print(envelope)

# img_folder = dl_folder
all_tiff = glob.glob(f"{dl_folder}/*.tiff", recursive=False)

options = {
    "format": "GTiff",
    "xRes": 10,
    "yRes": 10,
    "outputBoundsSRS": extentSRS,
    "outputBounds": envelope,
    #"cutlineDSName": extent,
    #"cropToCutline": True,
    "dstSRS": extentSRS+"-tap",
    "resampleAlg": "near",
}

for i, img in enumerate(all_tiff):
    print(i+1, "/", len(all_tiff))
    input_raster = gdal.Open(img)
    pre, ext = os.path.splitext(img)
    output_raster = pre + "_aligned"+ ext

    ds_aligned = gdal.Warp(output_raster, input_raster, **options)
    # print(ds_aligned.GetGeoTransform())
    print(output_raster)
    del ds_aligned